#### Criando insights em case com dados de uma empresa de fibra óptica fictícia

De inicio começo importando as bibliotecas para o tratamento dos dados e criação de insights

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re

Carregando a "amostra_360" para o df

In [ ]:
df = pd.read_excel('amostra_360.xlsx')

Visualizando o df

In [ ]:
df.head()

In [ ]:
df.info() #Solicitando informações do df

In [ ]:
df.isnull().sum() #Consultando celulas vazias por coluna

In [ ]:
df.dropna(axis = 1, how = 'all') #Removendo colunas completamente vazias

#### 54 colunas foram removidas.

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(axis = 1, thresh = 11) #thresh para remover colunas com apenas 11 numeros

Muitos dados em branco e fica dificil utilização da função "dropna". Sendo assim crio um df com colunas relevante para seguir com o tratamento de dados.

In [ ]:
list_columns = ['cd_tipo_survey_hp', 'ic_venda', 'velocidade_hc', 'ds_uf_hc', 'ds_municipio_hc', 'prestadora_servico', 'produto', 'dt_solicitacao', 'dt_instalacao', 'status_ordem_instalacao']
df_analytics = df[list_columns]
df_analytics

In [ ]:
df_analytics.isnull().sum()


## Tratamento de Dados

Como visto na função `isnull().sum()`, ainda temos algumas células vazias. Agora, preciso explicar as hipóteses levantadas que justificaram o tratamento dos dados por meio da exclusão das linhas com células vazias e a aplicação de um filtro para o status de instalação `"COMPLETED"`.

### Hipótese 1
No pior dos casos, se considerarmos que as linhas não convergem entre si, ainda assim o número de células vazias é muito menor que o número total de linhas. Portanto, foi considerado que sua exclusão não impacta significativamente a análise das amostras. Além disso, completar esses dados com valores aleatórios ou fazer a interpolação seria um processo trabalhoso que, provavelmente, não traria resultados tão significativos.

### Hipótese 2
Não posso considerar os dados de empresas que não possuem o status de instalação `"COMPLETED"`. Dessa forma, registros com status `"CANCELLED"` são descartados, pois o serviço não foi prestado, e o status `"IN_PROGRESS"` não me garante que o serviço não será cancelado antes de sua conclusão, como, por exemplo, a equipe chegar ao local para a instalação.

---

Com isso, optei por excluir as linhas com células vazias e aplicar um filtro na coluna `Status_Instalação` para manter apenas os registros com o status `"PLETED"`.




In [ ]:
df_analytics = df_analytics.dropna(axis=0)#Remolvo as linhas vazias que contem celulas vazias
df_analytics

In [ ]:
df_analytics.isnull().sum()

In [ ]:
#filtando o df apenas para o status de ordem "completo"
df_analytics = df_analytics.loc[df_analytics['status_ordem_instalacao']=='COMPLETED']
df_analytics

In [ ]:
#Usando a função rename para renomear as colunas do df_analytics
df_analytics = df_analytics.rename(columns={"cd_tipo_survey_hp": "Tipo_Local", "ic_venda": "Num_Vendas", "velocidade_hc": "Velocidade", "ds_uf_hc": "Estado", "ds_municipio_hc": "Municipio", "prestadora_servico": "Servico", "produto": "Produto", "dt_solicitacao": "Dt_Solicitacao", "dt_instalacao": "Dt_Instalacao", "status_ordem_instalacao": "Status_Instalacao"})
df_analytics

In [ ]:
df_analytics.reset_index(drop=True, inplace=True)#resetando o index de cada linhas

In [ ]:
df_analytics

In [ ]:
df_analytics.info()#agora verifico o dtype de cada coluna

In [ ]:
#transformando o dtype da coluna "Dt_Instalacao" de object para datetime
df_analytics["Dt_Instalacao"] = pd.to_datetime(df_analytics["Dt_Instalacao"])

In [ ]:
df_analytics.info()

In [ ]:
#criando coluna de duração para concluir o serviço
df_analytics['Duracao_Servico'] = df_analytics['Dt_Instalacao']-df_analytics['Dt_Solicitacao']

In [ ]:
df_analytics.info()

In [ ]:
#transformo essa nova coluna de timedelta para int
df_analytics['Duracao_Servico'] = df_analytics['Duracao_Servico'].dt.days.astype(int)

In [ ]:
df_analytics.info()

In [ ]:
df_analytics

In [ ]:
#observando a df_analytics a velocidade esta em object (valor da velocidade e sua unidade) e alguns dos dados estão escritos de forma diferente
#Sendo assim, uso a função apply em conjunto com lambda para substituir strigs por um espaço, tendo so numero nas celulas as transformo int
df_analytics['Velocidade'] = df_analytics['Velocidade'].apply(lambda x: re.sub('[^0-9]', '', str(x))).astype(int)

In [ ]:
df_analytics.info()

In [ ]:
#renomeio a coluna de velocidade, adicionando a unidade MBPS
df_analytics = df_analytics.rename(columns={'Velocidade': 'Velocidade_MBPS'})
df_analytics

In [ ]:
#Aqui percebo que para analisar os dados em mes e ano, preciso extrair esses dados da coluna "Dt_Solicitação"
#Sendo assim, tenho mes e ano em duas novas colunas
df_analytics['Mes'] = df_analytics['Dt_Solicitacao'].dt.strftime('%B')
df_analytics['Ano'] = df_analytics['Dt_Solicitacao'].dt.year
df_analytics

#### Com o DataFrame tratado e pronto para gerar os insights, começo a utilizar as bibliotecas `matplotlib` e `plotly.express`.
#### Observe que, neste ponto, procuro gerar os insights da forma mais variada possível. Isso é proposital, com o intuito de demonstrar o conhecimento em diversas funções das respectivas bibliotecas


### Insights para visulizar as vendas de acordo com o tempo.

In [ ]:

meses_vendas = df_analytics.groupby(['Ano', 'Mes'])['Num_Vendas'].sum().reset_index()

px.bar(meses_vendas, x='Mes', y='Num_Vendas', color= 'Ano').update_layout(title='Vendas Mensais por Ano').update_xaxes(title_text=None).update_yaxes(title_text=None)


#### Aqui é percepitivel que o mês de agosto, junho e julho de 2022 se destacou nas vendas. Por que será? O fatos desses meses fazerem parte da estação do ano de inverno. Pode ser uma justificativa, as pessoas passam mais o fim de semana em suas casas, aquelas que não tem internet porque passa pouquissimo tempo em casa e se satisfaz com os dados moveis, começam a necessitar de internet ? 
#### Outro ponto que chama atenção é a minima de vendas no mês de fevereiro, será que o verão contribui de forma oposta ao inverno?


In [ ]:
ano_vendas = df_analytics.groupby('Ano')['Num_Vendas'].sum()

colors = ['gray', 'yellow']

px.pie(ano_vendas, names = ano_vendas.index, values = ano_vendas.values, color_discrete_sequence=colors).update_layout(title='Vendas por Ano')

#### Por falta de dados não há como comparar os dados de 2022 e 2023.

#### Porém, apenas os dois primeiros meses de 2023 esta bem proximo de 2/12 do ano de 2022. O que leva a seguinte pergunta, nesse ritimo será que as vendas no ano de 2023 vão superar o ano anterior? 

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
colors = ['gray']

data_vendas = df_analytics.groupby('Dt_Solicitacao')['Num_Vendas'].sum()

ax.plot( data_vendas.index, data_vendas.values, color=colors[0], linewidth=2.0)
plt.show()

#### Aqui temos um grafico de linhas das vendas ao longo dos dias. Percebe-se uma diminuição no número de vendas por dia.

In [ ]:
#Gerando grafico de vendas por estado
vendas_por_estado = df_analytics.groupby('Estado')['Num_Vendas'].sum()

vendas_por_estado.plot.bar(color='gray');

In [ ]:
#Gerando grafico de duração de instalação por estado
fig, ax = plt.subplots(figsize=(8,5))

mean_duracao_estado = df_analytics.groupby('Estado')['Duracao_Servico'].mean()

mean_duracao_estado.plot.bar(color='gray', title = 'Duração de instalação(dias) por estado');

#### Nos gráficos de barras dos estados por vendas e duração de instalação. Percebe-se que nas regiões que temos um maior tempo de instalação, há uma deficiência de vendas.

#### Buscar parcerias com outros serviços do estado seria uma solução? Empresa que dispõe uma maior atenção aos clientes se destacam no mercado, agilidade na instação desso produto pode contribuir para uma maior fama na região consequentemente maior número de vendas.

In [ ]:
vendas_por_local = df_analytics.groupby('Tipo_Local')['Num_Vendas'].sum()

colors = ['gray', 'yellow']

px.pie(vendas_por_local, names = vendas_por_local.index, values = vendas_por_local.values, color_discrete_sequence=colors).update_layout(title='Vendas por tipo de local')

#### No gráfico de vendas por tipo de local, é notório uma dominância das vendas para moradia. E por mais que seja um grafico simples quer dizer muito. Com ele podemos obeserva o publico alvo das vendas. Direcionando assim estrategias de marketing para potencializar as vendas dos produtos, investindo em canais de marketing específicos e atigindo o público alvo.

In [ ]:
vendas_por_vel = df_analytics.groupby('Velocidade_MBPS')['Num_Vendas'].sum()
vendas_por_vel.plot.bar(color='gray', title='Vendas por pacote de velocidade');

#### Observe que o pacote de velocidade se destaca no número de vendas, provavelmente deve ser um pacote que atende as necessidades do cliente, o famoso custo beneficio. Aplicar esse pacote para outras regiões, a qual ele não esteja disponivel, ou tentativas de baratear os serviços de instalaçao podem potencializar as vendas.

In [ ]:
#Gerando grafico de duração de instalação por serviço
fig, ax = plt.subplots(figsize=(8,5))

mean_duracao_estado = df_analytics.groupby('Servico')['Duracao_Servico'].mean()

mean_duracao_estado.plot.bar(color='gray', title = 'Duração de instalação(dias) por Servico')

In [ ]:
vendas_por_servico = df_analytics.groupby('Servico')['Num_Vendas'].sum()

px.pie(vendas_por_servico, names = vendas_por_servico.index, values = vendas_por_servico.values).update_layout(legend=dict(title='Vendas por Serviço', orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

#### Como dito ateriormente, prestação de serviço rápido passa ao cliente uma visão de atenção, ele se sente especial. E os dados não dizem o contrario. Observe a empresa prestadora de serviço "Serede" no gráfico pizza perceba como ela se destaca no número de vendas, sendo responsavel por quase 40% das vendas. Agora olhe o gráfico anterior de media de duração de instalação por serviço, justamente a empresa que se destaca é a SEREDE, com a menor média de tempo para prestar serviço

In [ ]:
vendas_servico_estado = df_analytics.groupby(['Estado', 'Servico'])['Num_Vendas'].sum().reset_index()

px.bar(vendas_servico_estado, x = 'Estado', y = 'Num_Vendas', color = 'Servico', title = 'Vendas dos serviços por estados')

#### Nesse dois graficos, foi filtrado as vendas por serviços e as vendas por serviços nos estados. Sendo graficos que chamam muita atenção visto que, atitudes simples levam a grande resultando. Por exemplo,  buscar parcerias com as empresas Telemont, Conceta,  Serede e outras em outras regiões pode potencializar as vendas. Não só isso, pode ser feito parcerias com empresas com culturas parecidas com as de destaque. 